## Model build

In [1]:
import tensorflow as tf
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import os
from torch import tensor
# from torchmetrics.audio import ScaleInvariantSignalDistortionRatio
import concurrent.futures
import tqdm

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D,Conv2DTranspose, LeakyReLU, Dropout, concatenate, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models

In [3]:
if tf.config.list_physical_devices('GPU'):
    print("Using GPU for Training")
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0],True)
else:
    print("Using CPU for Training")

Using GPU for Training


In [4]:
import tensorflow as tf
import tensorflow.keras.backend as K

#definig the convolution block that consist of 2 conv layers.

def conv_block(input,no_filter,f_size,BN = False,drop_out = 0):
    x = tf.keras.layers.Conv2D(no_filter,f_size, padding = "same")(input)
    x = LeakyReLU(alpha=0.2)(x)
    if BN:
        x = tf.keras.layers.BatchNormalization()(x)
    
    residual = x
    x = tf.keras.layers.Conv2D(no_filter,f_size,padding = "same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    if BN :
        x = tf.keras.layers.BatchNormalization()(x)
    
    if drop_out > 0 :
        x = tf.keras.layers.Dropout(drop_out)(x)
    x = tf.keras.layers.Add()([x, residual])
    return x


def att_block(x, g, desired_dimensionality):
    x_shape = x.shape
    g_shape = g.shape
    
    
    # strides for xl should be 2 to equal the shapes before addition
    xl = tf.keras.layers.Conv2D(desired_dimensionality, (1, 1),strides=(2,2), padding="same")(x)
    xl = LeakyReLU(alpha=0.2)(xl)
    
    gl = tf.keras.layers.Conv2D(desired_dimensionality, (1, 1), padding="same")(g)
    gl = LeakyReLU(alpha=0.2)(gl)
    
    xg = tf.keras.layers.Add()([xl, gl])
    xg = LeakyReLU(alpha=0.2)(xg)
    xg = tf.keras.layers.Conv2D(1, (1, 1), activation="sigmoid", padding="same")(xg)
    xg_shape = xg.shape
    # Transpose xg to match the shape of x
    xg = tf.keras.layers.Conv2DTranspose(1, (x_shape[1], x_shape[2]), strides=(x_shape[1] // xg_shape[1], x_shape[2] // xg_shape[2]), padding='same')(xg)
    
    output = tf.keras.layers.Multiply()([xg, x])
    return output


def build_generator(no_filter, input_shape=(224, 224, 3)):
    # down-sampling process
    inputs = tf.keras.layers.Input(input_shape, dtype=tf.float32)
    x1 = conv_block(inputs, no_filter, (3,3), BN=True, drop_out=0.5)
    pool1 = tf.keras.layers.MaxPooling2D(2, 2)(x1)

    x2 = conv_block(pool1, 2 * no_filter, (3,3), BN=True, drop_out=0.5)
    pool2 = tf.keras.layers.MaxPooling2D(2, 2)(x2)

    x3 = conv_block(pool2, 4 * no_filter, (3,3), BN=True, drop_out=0.5)
    pool3 = tf.keras.layers.MaxPooling2D(2, 2)(x3)

    x4 = conv_block(pool3, 8 * no_filter, (3,3), BN=True, drop_out=0.5)
    pool4 = tf.keras.layers.MaxPooling2D(2, 2)(x4)

    # bottle-neck
    x5 = conv_block(pool4, 16 * no_filter, (3,3), BN=True, drop_out=0.5)

    # up-sampling layers

    x6 = att_block(x4, x5, no_filter * 2)
    u6 = tf.keras.layers.Conv2DTranspose(8 * no_filter,(3,3),strides=(2,2) ,padding='same')(x5)
    concate1 = tf.keras.layers.Concatenate()([x6, u6])
    conv6 = conv_block(concate1, 8 * no_filter, (5,5), BN=True, drop_out=0.5)
    
    x7= att_block(x3,conv6, no_filter*2)
    u7= tf.keras.layers.Conv2DTranspose(4 * no_filter,(3,3),strides=(2,2), padding='same')(conv6) 
    concate2 = tf.keras.layers.Concatenate()([x7,u7])
    conv7 = conv_block(concate2,4*no_filter,(5,5),BN = True,drop_out = 0.5) 


    x8= att_block(x2,conv7, no_filter*2)
    u8= tf.keras.layers.Conv2DTranspose(2 * no_filter,(3,3),strides=(2,2), padding='same')(conv7) 
    concate3 = tf.keras.layers.Concatenate()([x8,u8])
    conv8 = conv_block(concate3,2*no_filter,(3,3),BN = True,drop_out = 0.5) 



    x9= att_block(x1,conv8, no_filter*2)
    u9= tf.keras.layers.Conv2DTranspose(no_filter,(3,3),strides=(2,2),padding='same')(conv8) 
    concate4 = tf.keras.layers.Concatenate()([x9,u9])
    conv9 = conv_block(concate4,no_filter,(3,3),BN = True,drop_out = 0.5) 
    


    conv_final = tf.keras.layers.Conv2D(1, kernel_size=(4,4),dilation_rate=(2, 2), strides=(1,1),activation='sigmoid', padding='same')(conv8)
    conv_final =tf.keras.layers.BatchNormalization(axis=3)(conv_final)
    conv_final = tf.keras.layers.Conv2DTranspose(1,(3,3),strides=(2,2), activation="sigmoid", padding='same')(conv_final)
    
    output = Multiply()([conv_final, inputs])

    return tf.keras.Model(inputs, output)


In [4]:
# def build_generator(input_shape):
#     # Encoder
#     inputs = Input(input_shape)
#     conv1 = Conv2D(16, (5, 5), strides=(2, 2), padding='same')(inputs)
#     conv1 = LeakyReLU(alpha=0.2)(conv1)
#     conv1 = Dropout(0.5)(conv1)

#     conv2 = Conv2D(32, (5, 5), strides=(2, 2), padding='same')(conv1)
#     conv2 = LeakyReLU(alpha=0.2)(conv2)
#     conv2 = Dropout(0.5)(conv2)

#     conv3 = Conv2D(64, (5, 5), strides=(2, 2), padding='same')(conv2)
#     conv3 = LeakyReLU(alpha=0.2)(conv3)
#     conv3 = Dropout(0.5)(conv3)

#     conv4 = Conv2D(128, (5, 5), strides=(2, 2), padding='same')(conv3)
#     conv4 = LeakyReLU(alpha=0.2)(conv4)
#     conv4 = Dropout(0.5)(conv4)

#     conv5 = Conv2D(256, (5, 5), strides=(2, 2), padding='same')(conv4)
#     conv5 = LeakyReLU(alpha=0.2)(conv5)
#     conv5 = Dropout(0.5)(conv5)

#     conv6 = Conv2D(512, (5, 5), strides=(2, 2), padding='same')(conv5)
#     conv6 = LeakyReLU(alpha=0.2)(conv6)
#     conv6 = Dropout(0.5)(conv6)

#     # Decoder
#     up1 = Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same')(conv6)
#     up1 = LeakyReLU(alpha=0.2)(up1)
#     merge1 = concatenate([conv5, up1], axis=3)  # Changed axis to concatenate along the channel dimension
#     merge1 = Dropout(0.5)(merge1)

#     up2 = Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same')(merge1)
#     up2 = LeakyReLU(alpha=0.2)(up2)
#     merge2 = concatenate([conv4, up2], axis=3)  # Changed axis to concatenate along the channel dimension
#     merge2 = Dropout(0.5)(merge2)

#     up3 = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same')(merge2)
#     up3 = LeakyReLU(alpha=0.2)(up3)
#     merge3 = concatenate([conv3, up3], axis=3)  # Changed axis to concatenate along the channel dimension
#     merge3 = Dropout(0.5)(merge3)

#     up4 = Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same')(merge3)
#     up4 = LeakyReLU(alpha=0.2)(up4)
#     merge4 = concatenate([conv2, up4], axis=3)  # Changed axis to concatenate along the channel dimension
#     merge4 = Dropout(0.5)(merge4)

#     up5 = Conv2DTranspose(16, (5, 5), strides=(2, 2), padding='same')(merge4)
#     up5 = LeakyReLU(alpha=0.2)(up5)
#     merge5 = concatenate([conv1, up5], axis=3)  # Changed axis to concatenate along the channel dimension
#     merge5 = Dropout(0.5)(merge5)

#     # Add the final convolution layer
#     conv = Conv2D(1, (4, 4), dilation_rate=(2, 2), activation='sigmoid', padding='same')(merge5)

#     output = Conv2DTranspose(1, (5, 5), strides=(2, 2), activation="sigmoid", padding='same')(conv)
#     output = Multiply()([output, inputs])

#     model = Model(inputs=inputs, outputs=output)
#     return model

In [5]:
# Discriminator
def build_discriminator(input_shape):
    model = models.Sequential()

    model.add(layers.Conv2D(16, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [6]:
# # Define the input shape for only autoencoder
# input_shape = (128, 64, 1)

# # Create the U-Net model
# model = build_generator(input_shape)


# # Print the model summary
# # model.summary()

In [7]:
# Define input shape for adversial autoencoder
input_shape = (128, 64, 1)

# Build generator and discriminator
generator = build_generator(16,input_shape=(128,64,1) )
discriminator = build_discriminator(input_shape)

# Build GAN model
discriminator.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy')
generator.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

In [8]:
generator.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 64, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 128, 64, 16)  416         ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 128, 64, 16)  0           ['conv2d[0][0]']                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 64, 16)  64         ['leaky_re_lu[0][0]']        

 dropout_3 (Dropout)            (None, 16, 8, 128)   0           ['batch_normalization_7[0][0]']  
                                                                                                  
 add_3 (Add)                    (None, 16, 8, 128)   0           ['dropout_3[0][0]',              
                                                                  'batch_normalization_6[0][0]']  
                                                                                                  
 max_pooling2d_3 (MaxPooling2D)  (None, 8, 4, 128)   0           ['add_3[0][0]']                  
                                                                                                  
 conv2d_8 (Conv2D)              (None, 8, 4, 256)    819456      ['max_pooling2d_3[0][0]']        
                                                                                                  
 leaky_re_lu_8 (LeakyReLU)      (None, 8, 4, 256)    0           ['conv2d_8[0][0]']               
          

 leaky_re_lu_17 (LeakyReLU)     (None, 16, 8, 32)    0           ['add_7[0][0]']                  
                                                                                                  
 conv2d_17 (Conv2D)             (None, 16, 8, 1)     33          ['leaky_re_lu_17[0][0]']         
                                                                                                  
 conv2d_transpose_2 (Conv2DTran  (None, 32, 16, 1)   513         ['conv2d_17[0][0]']              
 spose)                                                                                           
                                                                                                  
 multiply_1 (Multiply)          (None, 32, 16, 64)   0           ['conv2d_transpose_2[0][0]',     
                                                                  'add_2[0][0]']                  
                                                                                                  
 conv2d_tr

 batch_normalization_18 (BatchN  (None, 64, 32, 1)   4           ['conv2d_30[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_transpose_8 (Conv2DTran  (None, 128, 64, 1)  26          ['batch_normalization_18[0][0]'] 
 spose)                                                                                           
                                                                                                  
 multiply_4 (Multiply)          (None, 128, 64, 1)   0           ['conv2d_transpose_8[0][0]',     
                                                                  'input_1[0][0]']                
                                                                                                  
Total params: 5,991,253
Trainable params: 5,988,371
Non-trainable params: 2,882
_____________________________

In [9]:
discriminator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 128, 64, 16)       160       
                                                                 
 batch_normalization_19 (Bat  (None, 128, 64, 16)      64        
 chNormalization)                                                
                                                                 
 re_lu (ReLU)                (None, 128, 64, 16)       0         
                                                                 
 conv2d_32 (Conv2D)          (None, 64, 32, 32)        4640      
                                                                 
 batch_normalization_20 (Bat  (None, 64, 32, 32)       128       
 chNormalization)                                                
                                                                 
 re_lu_1 (ReLU)              (None, 64, 32, 32)        0

## Data Prepare

In [10]:
# Function to load data from a single .mat file
def load_data(file_path):
    loaded_mat = scipy.io.loadmat(file_path)
    return loaded_mat['Segment_clean']  # Replace with the actual variable name

# Define the folder containing the .mat files
mat_folder = 'data/dataformodel_noaug/clean_stft_wgn0db'  # Replace with the actual path

# Initialize an empty list to store loaded data
loaded_data = []

# List all .mat files in the folder
mat_files = [os.path.join(mat_folder, filename) for filename in os.listdir(mat_folder) if filename.endswith('.mat')]

# Use concurrent processing to load data from multiple files simultaneously
with concurrent.futures.ThreadPoolExecutor() as executor:
    loaded_data = list(tqdm.tqdm(executor.map(load_data, mat_files)))

# Concatenate all loaded data into a single variable
clean_data = np.stack(loaded_data, axis=0)

4213it [00:02, 1856.76it/s]


In [11]:
clean_data.shape,np.std(np.real(clean_data))

((4213, 128, 64), 0.8289681539628687)

In [12]:
clean_real = np.real(clean_data)
clean_img  = np.imag(clean_data)
clean_real.shape,clean_img.shape

((4213, 128, 64), (4213, 128, 64))

In [13]:
# from sklearn.preprocessing import StandardScaler

# original_shape_clean = clean_real.shape
# clean_data_2d = clean_real.reshape(-1, original_shape_clean[-1])

# # Initialize the StandardScaler
# scaler_clean = StandardScaler()

# # Fit the scaler on your data and transform it
# scaled_cleandata_2d = scaler_clean.fit_transform(clean_data_2d)

# # Reshape the scaled data back to its original shape
# scaled_clean_data = scaled_cleandata_2d.reshape(original_shape_clean)

In [14]:
# Function to load data from a single .mat file
def load_data(file_path):
    loaded_mat = scipy.io.loadmat(file_path)
    return loaded_mat['Segment_noisy']  # Replace with the actual variable name

# Define the folder containing the .mat file
mat_folder = 'data/dataformodel_noaug/noisy_stft_wgn0db'  # Replace with the actual path

# Initialize an empty list to store loaded data
loaded_data = []

# List all .mat files in the folder
mat_files = [os.path.join(mat_folder, filename) for filename in os.listdir(mat_folder) if filename.endswith('.mat')]

# Use concurrent processing to load data from multiple files simultaneously
with concurrent.futures.ThreadPoolExecutor() as executor:
    loaded_data = list(tqdm.tqdm(executor.map(load_data, mat_files)))

# Concatenate all loaded data into a single variable
noisy_data = np.stack(loaded_data, axis=0)

4213it [00:02, 1810.57it/s]


In [15]:
noisy_data.shape,np.std(np.real(noisy_data[0]))

((4213, 128, 64), 1.2157565236630237)

In [16]:
noisy_real = np.real(noisy_data)
noisy_img  = np.imag(noisy_data)
noisy_real.shape,noisy_img.shape

((4213, 128, 64), (4213, 128, 64))

In [17]:
# original_shape = noisy_real.shape
# noisy_data_2d = noisy_real.reshape(-1, original_shape[-1])

# # Initialize the StandardScaler
# scaler = StandardScaler()

# # Fit the scaler on your data and transform it
# scaled_data_2d = scaler.fit_transform(noisy_data_2d)

# # Reshape the scaled data back to its original shape
# scaled_data = scaled_data_2d.reshape(original_shape)

In [18]:
from sklearn.model_selection import train_test_split
X_trn, X_valid, y_trn, y_valid = train_test_split(noisy_real, clean_real, test_size=0.1, random_state=42)

In [19]:
X_trn.shape, X_valid.shape, y_trn.shape, y_valid.shape

((3791, 128, 64), (422, 128, 64), (3791, 128, 64), (422, 128, 64))

## Model Run

In [20]:
X_train_noisy = X_trn
y_train_clean =y_trn

In [21]:
X_train_noisy.shape,y_train_clean.shape

((3791, 128, 64), (3791, 128, 64))

In [22]:
import time

# Training loop
epochs = 200
batch_size = 64

total_batches = int(np.ceil(len(X_train_noisy) / batch_size))

# Create a directory to save the models
save_dir = "saved_mode"
os.makedirs(save_dir, exist_ok=True)

for epoch in range(epochs):
    start_time = time.time()
    print(f"Epoch {epoch + 1}/{epochs}", end=' ')

    for i in range(0, len(X_train_noisy), batch_size):
        current_batch_size = min(batch_size, len(X_train_noisy) - i)

        # Train the discriminator
        real_data = y_train_clean[i:i + current_batch_size]
        real_labels = tf.ones((current_batch_size, 1))
        d_loss_real = discriminator.train_on_batch(real_data, real_labels)

        generated_data = generator.predict(X_train_noisy[i:i + current_batch_size])
        generate_labels = tf.zeros((current_batch_size, 1))
        d_loss_generate = discriminator.train_on_batch(generated_data, generate_labels)

        # Calculate total discriminator loss
        d_loss = 0.5 * np.add(d_loss_real, d_loss_generate)

        # Train the generator
        g_loss = generator.train_on_batch(X_train_noisy[i:i + current_batch_size], y_train_clean[i:i + current_batch_size])

        print(f"\rEpoch {epoch + 1}/{epochs}, {i//batch_size + 1}/{total_batches} - D Loss: {d_loss}, G Loss: {g_loss}", end='')
        
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f" - {elapsed_time:.2f} s")
    
    # Save the model starting from epoch 1500 and every 500 epochs thereafter
    if (epoch + 1) >= 100 and (epoch + 1) % 50 == 0:
        model_name = f'model_{epoch + 1}.h5'
        model_path = os.path.join(save_dir, model_name)
        generator.save(model_path)
        print(f"Saved model at epoch {epoch + 1} to {model_path}")



Epoch 1/3000, 60/60 - D Loss: 0.001561547358505777, G Loss: 0.4643370509147644 - 25.78 s
Epoch 2/3000, 60/60 - D Loss: 0.0005540987840504386, G Loss: 0.43324682116508484 - 17.43 s
Epoch 3/3000, 60/60 - D Loss: 0.0002852453053492354, G Loss: 0.4276280105113983 - 17.50 s
Epoch 4/3000, 60/60 - D Loss: 0.00017424047837266698, G Loss: 0.42362621426582336 - 17.57 s
Epoch 5/3000, 60/60 - D Loss: 0.00011885871572303586, G Loss: 0.4194992780685425 - 17.69 s
Epoch 6/3000, 60/60 - D Loss: 8.539981990907108e-05, G Loss: 0.4159187376499176 - 17.87 s
Epoch 7/3000, 60/60 - D Loss: 6.318226587609388e-05, G Loss: 0.4107545018196106 - 17.65 s
Epoch 8/3000, 60/60 - D Loss: 4.9804860282165464e-05, G Loss: 0.4055953621864319 - 17.39 s
Epoch 9/3000, 60/60 - D Loss: 4.006914105048054e-05, G Loss: 0.4029640853404999 - 17.48 s
Epoch 10/3000, 60/60 - D Loss: 3.3823002013377845e-05, G Loss: 0.3979806900024414 - 17.69 s
Epoch 11/3000, 60/60 - D Loss: 2.8674054647126468e-05, G Loss: 0.3952995538711548 - 17.60 s
Ep

Epoch 178/3000, 60/60 - D Loss: 1.7044992084436217e-09, G Loss: 0.23039166629314423 - 17.74 s
Epoch 179/3000, 60/60 - D Loss: 1.694609036428929e-09, G Loss: 0.23357197642326355 - 17.62 s
Epoch 180/3000, 60/60 - D Loss: 1.7081950159925086e-09, G Loss: 0.227168008685112 - 17.62 s
Epoch 181/3000, 60/60 - D Loss: 1.667303697883149e-09, G Loss: 0.22446492314338684 - 17.59 s
Epoch 182/3000, 60/60 - D Loss: 1.6555378734572912e-09, G Loss: 0.22698785364627838 - 17.66 s
Epoch 183/3000, 60/60 - D Loss: 1.7097945004262982e-09, G Loss: 0.22801168262958527 - 17.67 s
Epoch 184/3000, 60/60 - D Loss: 1.7617305531070215e-09, G Loss: 0.2297397404909134 - 17.67 s
Epoch 185/3000, 60/60 - D Loss: 1.6875293662454993e-09, G Loss: 0.22520847618579865 - 17.88 s
Epoch 186/3000, 60/60 - D Loss: 1.6768699345748317e-09, G Loss: 0.2255282700061798 - 17.57 s
Epoch 187/3000, 60/60 - D Loss: 1.5811471160143142e-09, G Loss: 0.2242555469274521 - 17.25 s
Epoch 188/3000, 60/60 - D Loss: 1.701247337448919e-09, G Loss: 0.22

Epoch 352/3000, 60/60 - D Loss: 2.510291946533627e-09, G Loss: 0.20002005994319916 - 17.84 s
Epoch 353/3000, 60/60 - D Loss: 2.2796830048688044e-09, G Loss: 0.19919632375240326 - 17.66 s
Epoch 354/3000, 60/60 - D Loss: 2.622737631241545e-09, G Loss: 0.20671378076076508 - 17.87 s
Epoch 355/3000, 60/60 - D Loss: 2.645781919941026e-09, G Loss: 0.20028747618198395 - 17.77 s
Epoch 356/3000, 60/60 - D Loss: 2.3897441595033264e-09, G Loss: 0.20320835709571838 - 18.15 s
Epoch 357/3000, 60/60 - D Loss: 1.941245400538527e-09, G Loss: 0.1994580626487732 - 17.78 s
Epoch 358/3000, 60/60 - D Loss: 2.2981224701013048e-09, G Loss: 0.199344664812088 - 17.49 s
Epoch 359/3000, 60/60 - D Loss: 2.51871478534893e-09, G Loss: 0.19847173988819122 - 17.48 s
Epoch 360/3000, 60/60 - D Loss: 2.809868275555072e-09, G Loss: 0.2004987597465515 - 17.65 s
Epoch 361/3000, 60/60 - D Loss: 2.5637388531563055e-09, G Loss: 0.20475246012210846 - 17.64 s
Epoch 362/3000, 60/60 - D Loss: 2.6632915109403577e-09, G Loss: 0.20924

Epoch 440/3000, 60/60 - D Loss: 3.2038100783449153e-09, G Loss: 0.19521383941173553 - 17.72 s
Epoch 441/3000, 60/60 - D Loss: 3.5985900231905177e-09, G Loss: 0.1948288530111313 - 17.63 s
Epoch 442/3000, 60/60 - D Loss: 3.0467089059960095e-09, G Loss: 0.19790984690189362 - 17.53 s
Epoch 443/3000, 60/60 - D Loss: 3.0757678001758215e-09, G Loss: 0.19592711329460144 - 17.72 s
Epoch 444/3000, 60/60 - D Loss: 3.554776709102647e-09, G Loss: 0.19810204207897186 - 17.65 s
Epoch 445/3000, 60/60 - D Loss: 2.484684000214621e-09, G Loss: 0.19690735638141632 - 17.55 s
Epoch 446/3000, 60/60 - D Loss: 3.942171793863691e-09, G Loss: 0.19866636395454407 - 17.58 s
Epoch 447/3000, 60/60 - D Loss: 3.842301359024969e-09, G Loss: 0.19660472869873047 - 18.00 s
Epoch 448/3000, 60/60 - D Loss: 3.977376386576914e-09, G Loss: 0.19795511662960052 - 17.40 s
Epoch 449/3000, 60/60 - D Loss: 2.8586683228915e-09, G Loss: 0.19557200372219086 - 17.55 s
Epoch 450/3000, 60/60 - D Loss: 2.7000494964846133e-09, G Loss: 0.197

Epoch 616/3000, 60/60 - D Loss: 6.850539874952988e-09, G Loss: 0.19301535189151764 - 17.67 s
Epoch 617/3000, 60/60 - D Loss: 3.841800579051924e-09, G Loss: 0.192821204662323 - 17.90 s
Epoch 618/3000, 60/60 - D Loss: 7.74368597369901e-09, G Loss: 0.19417144358158112 - 18.00 s
Epoch 619/3000, 60/60 - D Loss: 4.920736571684126e-09, G Loss: 0.191740483045578 - 17.91 s
Epoch 620/3000, 60/60 - D Loss: 5.396236561761025e-09, G Loss: 0.19280311465263367 - 17.65 s
Epoch 621/3000, 60/60 - D Loss: 6.653916926127601e-09, G Loss: 0.1922667771577835 - 17.74 s
Epoch 622/3000, 60/60 - D Loss: 7.694674845210026e-09, G Loss: 0.19363349676132202 - 17.88 s
Epoch 623/3000, 60/60 - D Loss: 6.523123870921799e-09, G Loss: 0.19133888185024261 - 17.71 s
Epoch 624/3000, 60/60 - D Loss: 4.84892174676399e-09, G Loss: 0.19237588346004486 - 17.66 s
Epoch 625/3000, 60/60 - D Loss: 5.631858784882482e-09, G Loss: 0.19256886839866638 - 17.73 s
Epoch 626/3000, 60/60 - D Loss: 5.417533439217026e-09, G Loss: 0.193359434604

...

Epoch 2348/3000, 60/60 - D Loss: 3.070909619395934e-05, G Loss: 0.18844643235206604 - 17.76 s
Epoch 2349/3000, 60/60 - D Loss: 2.3889044115321667e-05, G Loss: 0.18830230832099915 - 17.88 s
Epoch 2350/3000, 60/60 - D Loss: 2.0157386728669735e-05, G Loss: 0.189915731549263 - 17.61 s
Epoch 2351/3000, 60/60 - D Loss: 1.9206540287086682e-05, G Loss: 0.18951615691184998 - 17.84 s
Epoch 2352/3000, 60/60 - D Loss: 1.4773372498666504e-05, G Loss: 0.18828897178173065 - 17.63 s
Epoch 2353/3000, 60/60 - D Loss: 1.8032500605613677e-05, G Loss: 0.1889525204896927 - 17.76 s
Epoch 2354/3000, 60/60 - D Loss: 1.819863834384705e-05, G Loss: 0.18828894197940826 - 17.57 s
Epoch 2355/3000, 60/60 - D Loss: 1.104738633728175e-05, G Loss: 0.18907544016838074 - 17.66 s
Epoch 2356/3000, 60/60 - D Loss: 7.946894797328241e-06, G Loss: 0.18853794038295746 - 17.78 s
Epoch 2357/3000, 60/60 - D Loss: 7.613549897200755e-06, G Loss: 0.1884279102087021 - 17.70 s
Epoch 2358/3000, 60/60 - D Loss: 6.388396634804394e-06, G L

Epoch 2521/3000, 60/60 - D Loss: 1.8427290112376076e-10, G Loss: 0.1887187361717224 - 17.37 s
Epoch 2522/3000, 60/60 - D Loss: 1.821462127448662e-10, G Loss: 0.18770958483219147 - 17.32 s
Epoch 2523/3000, 60/60 - D Loss: 2.232283597742744e-10, G Loss: 0.18911303579807281 - 17.57 s
Epoch 2524/3000, 60/60 - D Loss: 2.1193616122055364e-10, G Loss: 0.1889965683221817 - 17.47 s
Epoch 2525/3000, 60/60 - D Loss: 1.9472106955689293e-10, G Loss: 0.18777327239513397 - 17.79 s
Epoch 2526/3000, 60/60 - D Loss: 2.0364992466903944e-10, G Loss: 0.18789604306221008 - 17.95 s
Epoch 2527/3000, 60/60 - D Loss: 2.2511165353393103e-10, G Loss: 0.1902753859758377 - 17.92 s
Epoch 2528/3000, 60/60 - D Loss: 2.718397557971647e-10, G Loss: 0.1878253072500229 - 17.72 s
Epoch 2529/3000, 60/60 - D Loss: 2.306682271813144e-10, G Loss: 0.18795450031757355 - 18.23 s
Epoch 2530/3000, 60/60 - D Loss: 1.786938044230765e-10, G Loss: 0.19169673323631287 - 17.60 s
Epoch 2531/3000, 60/60 - D Loss: 1.995478293305539e-10, G L

Epoch 2608/3000, 60/60 - D Loss: 2.1862359522745872e-06, G Loss: 0.1886233389377594 - 17.38 s
Epoch 2609/3000, 60/60 - D Loss: 2.0905137388697523e-06, G Loss: 0.188328817486763 - 17.56 s
Epoch 2610/3000, 60/60 - D Loss: 2.0189964065279314e-06, G Loss: 0.18841847777366638 - 17.75 s
Epoch 2611/3000, 60/60 - D Loss: 1.9638347364516265e-06, G Loss: 0.18796110153198242 - 17.71 s
Epoch 2612/3000, 60/60 - D Loss: 1.934202618611436e-06, G Loss: 0.18831388652324677 - 17.50 s
Epoch 2613/3000, 60/60 - D Loss: 1.8896701760289653e-06, G Loss: 0.18882662057876587 - 17.84 s
Epoch 2614/3000, 60/60 - D Loss: 1.841115176239895e-06, G Loss: 0.1875966340303421 - 17.72 s
Epoch 2615/3000, 60/60 - D Loss: 1.8563314384510698e-06, G Loss: 0.18801210820674896 - 17.48 s
Epoch 2616/3000, 60/60 - D Loss: 1.8162697200865136e-06, G Loss: 0.19251154363155365 - 17.56 s
Epoch 2617/3000, 60/60 - D Loss: 1.707139106471231e-06, G Loss: 0.18896891176700592 - 17.63 s
Epoch 2618/3000, 60/60 - D Loss: 1.6666814638228653e-06, 

Epoch 2782/3000, 60/60 - D Loss: 2.212944181678722e-06, G Loss: 0.18736211955547333 - 17.79 s
Epoch 2783/3000, 60/60 - D Loss: 2.079314878500327e-06, G Loss: 0.18849973380565643 - 17.71 s
Epoch 2784/3000, 60/60 - D Loss: 2.0154459576815498e-06, G Loss: 0.18754450976848602 - 17.72 s
Epoch 2785/3000, 60/60 - D Loss: 1.970982005161659e-06, G Loss: 0.18869170546531677 - 17.66 s
Epoch 2786/3000, 60/60 - D Loss: 2.0265538687169737e-06, G Loss: 0.18781164288520813 - 17.76 s
Epoch 2787/3000, 60/60 - D Loss: 1.945224124000333e-06, G Loss: 0.18760773539543152 - 17.72 s
Epoch 2788/3000, 60/60 - D Loss: 1.8913697310551925e-06, G Loss: 0.18770574033260345 - 18.06 s
Epoch 2789/3000, 60/60 - D Loss: 1.566176770495531e-06, G Loss: 0.18715573847293854 - 17.55 s
Epoch 2790/3000, 60/60 - D Loss: 1.6081873681628167e-06, G Loss: 0.18596665561199188 - 17.83 s
Epoch 2791/3000, 60/60 - D Loss: 1.6446115722601462e-06, G Loss: 0.188058540225029 - 17.75 s
Epoch 2792/3000, 60/60 - D Loss: 1.5767681169620573e-06, 

Epoch 2956/3000, 60/60 - D Loss: 4.829917057191176e-10, G Loss: 0.18752281367778778 - 17.59 s
Epoch 2957/3000, 60/60 - D Loss: 4.0153764138350597e-10, G Loss: 0.18843625485897064 - 17.76 s
Epoch 2958/3000, 60/60 - D Loss: 3.840925172215331e-10, G Loss: 0.18851333856582642 - 17.60 s
Epoch 2959/3000, 60/60 - D Loss: 4.714592494174826e-10, G Loss: 0.18865086138248444 - 17.73 s
Epoch 2960/3000, 60/60 - D Loss: 3.6072286139995924e-10, G Loss: 0.1886470913887024 - 17.71 s
Epoch 2961/3000, 60/60 - D Loss: 3.416602338960267e-10, G Loss: 0.18838991224765778 - 17.75 s
Epoch 2962/3000, 60/60 - D Loss: 4.0412792787574933e-10, G Loss: 0.18704207241535187 - 17.78 s
Epoch 2963/3000, 60/60 - D Loss: 5.691701482038903e-10, G Loss: 0.19090063869953156 - 17.85 s
Epoch 2964/3000, 60/60 - D Loss: 3.3495528893445753e-10, G Loss: 0.18953533470630646 - 17.71 s
Epoch 2965/3000, 60/60 - D Loss: 2.7626952320549346e-10, G Loss: 0.1872435212135315 - 17.76 s
Epoch 2966/3000, 60/60 - D Loss: 3.5214141773543147e-10, 

In [ ]:
fgtg

In [26]:
# # Train the model

# initial_learning_rate = 0.0001  # Define your initial learning rate 0.0001  best
# adam_optimizer = Adam(learning_rate=initial_learning_rate)

# model.compile(loss='mean_squared_error', optimizer=adam_optimizer,metrics="accuracy")

# model.fit(X_train_noisy,y_train_clean , epochs=3, batch_size=64)

In [23]:
pred = generator.predict(X_valid)

In [24]:
# pred = model.predict(X_valid)
10 * np.log10(np.sum(np.expand_dims(y_valid, axis=-1)**2) / np.sum((np.expand_dims(y_valid, axis=-1) - pred)**2))

6.576876194388265

In [25]:
# si_sdr = ScaleInvariantSignalDistortionRatio()
# target = tensor([np.expand_dims(y_valid,axis=-1)])
# preds = tensor([pred])
# si_sdr(preds, target)

In [ ]:
%aug data
9.9473 %wiithout generator
11.4689 %with generator 

%without aug
9.3017

In [26]:
generator.save("generator_3000_wgn0.h5")

In [ ]:
# p = generator.predict(np.expand_dims(X_train_noisy[2][np.newaxis, :, :], axis=-1))
# p= p.reshape((128, 64))
# scipy.io.savemat('New_AS_002_denoise_model0001_aug_cv250_gg.mat', {'denoise_spe':p})

## Prediction

In [70]:
from keras.models import load_model

# Load the model from the file
modell = load_model("generator_3000.h5")

In [71]:
# Create an empty list to store the denoised spectrograms
denoised_spectrograms = []

# Specify the directory containing the noisy data files
noisy_data_directory = 'test_noisy_data/'

# Loop through all files in the directory
for file_name in os.listdir(noisy_data_directory):
    if file_name.endswith('.mat'):
        # Load the noisy data from the MAT file
        file_path = os.path.join(noisy_data_directory, file_name)
        test_noisy_data = scipy.io.loadmat(file_path)['Segment_noisy']

        noisy_data_real = np.real(test_noisy_data)
        noisy_data_imag = np.imag(test_noisy_data)
        
        
        # Expand dimensions
        noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)
   

        # Apply the model to denoise the data
        denoised_data_real = modell.predict(noisy_data_real)

        denoised_data_real = denoised_data_real.reshape((128, 64))
        
        # Reshape to (128, 64)
        #denoised_data = np.vectorize(complex)(denoised_data_real, noisy_data_imag)
        

        # Append the denoised data to the list
        denoised_spectrograms.append(denoised_data_real)

# Stack the denoised spectrograms along a new axis (4D array)
merged_denoised_data = np.concatenate(denoised_spectrograms, axis=1)

In [72]:
merged_denoised_data.shape

(128, 320)

In [73]:
scipy.io.savemat('New_AS_002_denoise_model0001_aug_cv250.mat', {'denoise_spe': merged_denoised_data})

In [36]:
merged_denoised_data

array([[-2.5413128e-02,  7.7895157e-02,  1.9338399e-01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-2.9260597e-01,  2.1329612e-01, -5.5941917e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 4.7727647e-01, -8.1839895e-01,  1.8247497e-01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 2.0739461e-05,  1.1372420e-03, -1.6909323e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-3.3129403e-05, -1.3230698e-03, -1.5605466e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-2.8804841e-04, -5.4520206e-04,  1.0511472e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)